# 【例題2.8】

## (6) 洪水追跡計算
### ④Muskingum-Cunge法のパラメータ設定

In [1]:
'''Muskingum-Cunge法のパラメータ設定'''
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize
EPS_DEPTH = 1e-15

#### 流れの諸量を計算するクラス

In [2]:
class FlowParam:
    """流れの諸量を計算するクラス"""
    
    @staticmethod
    def calcA(_h, _B):
        """流積"""
        return(_h*_B)

    @staticmethod
    def calc_s(_h, _B):
        """潤辺"""
        return(_B+2.*_h)    
    
    @staticmethod
    def calcR(_A, _s):
        """径深"""
        return(_A/_s)
        
    @staticmethod
    def calcQ0(_n, _A, _R, _Ib):
        """等流時の流量を求める関数"""
        return(1./_n*_A*_R**(2./3.)*_Ib**.5)
    
    @staticmethod
    def __eqUniform(_h, _n, _B, _Ib, _Q):
        """等流の関係"""
        __A = FlowParam.calcA(_h, _B)
        __s = FlowParam.calc_s(_h, _B)
        __R = FlowParam.calcR(__A, __s)
        return(np.abs(-_Ib+_n**2.*(_Q/__A)**2./__R**(4./3.)))
        
    @staticmethod
    def calch0(_n, _B, _Ib, _Q):
        """等流水深を求める関数"""
        __h0 = scipy.optimize.fmin(FlowParam.__eqUniform, 
                                   x0=[(_n*_n*(_Q/_B)**2./_Ib)**(3./10.)], 
                                   xtol=EPS_DEPTH, 
                                   disp=False, args=(_n, _B, _Ib, _Q, ))[0]
        return(__h0)

#### (a) ピーク流量の平均伝播速度$L/T_p$
- 伝播速度$\lambda$の計算

In [20]:
# パラメータの設定
Ib= 1./2000.
B = 200.
n =0.03
# ピーク流量
Qp = 2000.0
# ピーク流量前後の流量
Q = np.array([1516.121116, Qp, 1601.393261])
# 流量算出地点
L = 100000.0
# 等流水深
h = FlowParam.calch0(n, B, Ib, Qp)
# 流速
U= Qp/(B*h)
print("等流流速：", U)
# 伝播速度
Lamb = 5./3.* U
print("伝播速度：",Lamb)

等流流速： 2.0663716601923086
伝播速度： 3.4439527669871812


- 平均伝播速度$L/T_p$の設定

In [21]:
# L/Tpを入力
L_Tp = 3.5

#### (b) 減衰パラメータ$\alpha_p = \frac{1}{2}(L/B)(1/I_b)$

In [22]:
alpha_p = .5*(L/B)*(1./Ib)
print(alpha_p)

500000.0


#### (c) 流量ピークの曲率 $|\frac{d^2Q_p}{\delta t^2}|=|\frac{Q_{-1}+Q_1-2Q_p}{\delta t^2}|$

- 流量ピークまでの時間$T_p$=15(hr)であるので，$\delta t$ = 7200(s)とする．($\delta t$は，$T_p/5$程度)

- $\delta t$の設定

In [23]:
deltaT = 7200.

In [24]:
def d2Qp_dt2(Q0, Q1, Q2,dT):
    return(np.abs((Q0+Q2-2.*Q1)/dT**2.))

d2Qp_dt2 = np.abs((Q[0]+Q[2]-2.*Q[1])/dT**2.)
print(d2Qp_dt2)

1.7023256616512348e-05


#### (d) $x$=0 ~ $L$の間のピーク流量の減衰量$\Delta Q = \frac{\alpha_p}{(L/T_p)^3}Q_p|\frac{d^2Q_p}{dt^2}|$

In [25]:
def calcdQ(alpha_p, L_Tp, Qp, _d2Qp_dt2):
    return(alpha_p/(L_Tp)**3.*Qp*_d2Qp_dt2)

dQ = calcdQ(alpha_p, L_Tp, Qp, d2Qp_dt2)
print(dQ)

397.0438860994133


####　(e) $\Delta Q_{new}$と$\omega$の計算

- $\Delta Q \le 0.1 Q_p$の場合：$\omega = L/T_p$

- $\Delta Q > 0.1 Q_p$の場合：$\Delta Q_{New} = Q_p\left\{1-\exp\left(-\frac{\Delta Q}{Q_p}\right)\right\}$; $\omega = \frac{L}{T_p}-\frac{2.\alpha_p}{L^2}\Delta Q_{New}$

In [26]:
def calcNewLambda(dQ, Qp, L_Tp, alpha_p, L):
    if(dQ <= 0.1*Qp):
        __lambda = L_Tp
    else:
        __dQNew = Qp*(1.-np.exp(-dQ/Qp))
        __lambda = L_Tp-2.*alpha_p/L/L*__dQNew
    print("dQNew, omega = ", __dQNew, __lambda)
    return(__lambda)

newLambda = calcNewLambda(dQ, Qp, L_Tp, alpha_p, L)
print(newLambda)

dQNew, omega =  360.1164429603982 3.46398835570396
3.46398835570396


#### (f) $\omega$と$\Delta Q_{new}$の収束計算

In [27]:
while(1):
    _tmpL = newLambda
    dQNew = calcdQ(alpha_p, newLambda, Qp, d2Qp_dt2)
    newLambda = calcNewLambda(dQNew, Qp, L_Tp, alpha_p, L)
    if(np.abs(_tmpL-newLambda)< EPS_DEPTH):
        break
print(newLambda)

dQNew, omega =  370.3436872843049 3.4629656312715693
dQNew, omega =  370.6394203966252 3.4629360579603374
dQNew, omega =  370.64797627526593 3.4629352023724733
dQNew, omega =  370.6482238097657 3.4629351776190234
dQNew, omega =  370.6482309713159 3.4629351769028682
dQNew, omega =  370.6482311785106 3.462935176882149
dQNew, omega =  370.64823118450494 3.4629351768815493
dQNew, omega =  370.64823118467837 3.462935176881532
dQNew, omega =  370.64823118468325 3.4629351768815315
3.4629351768815315


#### (g) 平均ピーク流量 $\bar{Q_p}$の推定：$\bar{Q_p} = Q_p-\frac{1}{2}\Delta Q_{New}$

In [28]:
barQp = Qp-.5*dQNew
print(barQp)

1795.0350673165794


#### (h) 係数$\mu$の推定：$\mu = \alpha_p\bar{Q_p}/L$

In [34]:
mu = alpha_p*barQp/L
print(mu)

8975.175336582897


#### (i) KとXの推定
- $K = \frac{\Delta x}{\omega}$
- $X= 0.5-\mu/(\omega \cdot dx)$


In [35]:
dx =10000.0
K = dx/newLambda
X= .5-mu/(newLambda*dx)
print(K, X)

2887.7237052428095 0.24082173421838954


####  (j) dTのチェック

- 水理公式集(昭和60年版　p.216 図4.5)から$X$に対する$\frac{\Delta x}{\omega dt}=a$を読み取る．
- $\Delta t > \Delta x /(a\omega)$を満たすように$\Delta t$を設定

In [31]:
# Xに対する値 水理公式集(昭和60年版　p.216 図4.5)
dx_wdt = 0.80

In [36]:
#  dTのチェック
__dT = dx/(dx_wdt*newLambda)
print(__dT)

3609.6546315535115


#### dTの設定

In [37]:
dT = 7200.
if(__dT > dT):
    print("Check dT")
else:
    print("OK", __dT, dT)

OK 3609.6546315535115 7200.0
